## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 16th Feb. 23rd


In [4]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-18-2025,Condo/Co-op,2700 N Federal Hwy #204,Boynton Beach,FL,33435.0,702500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/2700-N...,Beaches MLS,F10480745,N,Y,26.553696,-80.05355


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [9]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [12]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [13]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-18    25
2025-02-21    19
2025-02-20    13
2025-02-19     9
2025-02-17     2
Name: count, dtype: int64

In [14]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [15]:
df_top_ten = df2.head(10)

In [16]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
57,PAST SALE,2025-02-20,Condo/Co-op,1500 Beach Rd #203,Tequesta,FL,33469.0,8980000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Beaches MLS,RX-11003738,N,Y,26.961813,-80.079386
65,PAST SALE,2025-02-20,Condo/Co-op,130 Sunrise Ave Ph 8,Palm Beach,FL,33480.0,7905500.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Beaches MLS,RX-10977456,N,Y,26.719588,-80.036274
35,PAST SALE,2025-02-20,Condo/Co-op,2500 S Ocean Blvd #602,Boca Raton,FL,33432.0,6275000.0,6.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-11031676,N,Y,26.326626,-80.074533
5,PAST SALE,2025-02-18,Condo/Co-op,2720 Donald Ross Rd #505,Palm Beach Gardens,FL,33410.0,5600000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11052872,N,Y,26.881072,-80.073865
34,PAST SALE,2025-02-18,Condo/Co-op,2070 N Ocean Blvd Ph 4,Boca Raton,FL,33431.0,4020480.0,4.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Beaches MLS,RX-11035724,N,Y,26.369827,-80.068332
2,PAST SALE,2025-02-19,Condo/Co-op,3951 N Ocean Blvd #603,Gulf Stream,FL,33483.0,3400000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Beaches MLS,RX-11065165,N,Y,26.501091,-80.052356
32,PAST SALE,2025-02-21,Condo/Co-op,170 N Ocean Blvd Unit 510/512,Palm Beach,FL,33480.0,3384250.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11038081,N,Y,26.721061,-80.035205
16,PAST SALE,2025-02-19,Condo/Co-op,101 N Clematis St #402,West Palm Beach,FL,33401.0,2950000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/101-...,Beaches MLS,RX-11046613,N,Y,26.714014,-80.050436
11,PAST SALE,2025-02-18,Condo/Co-op,800 Ocean Dr #1004,Juno Beach,FL,33408.0,2007500.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/800-Ocean...,Beaches MLS,RX-11034227,N,Y,26.878233,-80.052627
66,PAST SALE,2025-02-18,Condo/Co-op,100 Sunrise Ave #219,Palm Beach,FL,33480.0,2000000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-10950177,N,Y,26.719562,-80.034847


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [17]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [18]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [19]:
response_df = pd.DataFrame(response_list)

In [20]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [21]:
df_top_ten = merged_df

In [22]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-20,Condo/Co-op,1500 Beach Rd #203,Tequesta,FL,33469.0,8980000.0,3.0,4.5,...,-80.079386,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Nicholas Peterson,Daniel Riordan,None,None,Ryan Billings,Water Pointe Realty Group,None,None
1,PAST SALE,2025-02-20,Condo/Co-op,130 Sunrise Ave Ph 8,Palm Beach,FL,33480.0,7905500.0,3.0,3.0,...,-80.036274,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Spencer Schlager,Douglas Elliman,None,None,Spencer Schlager,Douglas Elliman,None,None
2,PAST SALE,2025-02-20,Condo/Co-op,2500 S Ocean Blvd #602,Boca Raton,FL,33432.0,6275000.0,6.0,4.5,...,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None
3,PAST SALE,2025-02-18,Condo/Co-op,2720 Donald Ross Rd #505,Palm Beach Gardens,FL,33410.0,5600000.0,4.0,4.5,...,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Donna Lederman,Waterfront Properties & Club C,None,None
4,PAST SALE,2025-02-18,Condo/Co-op,2070 N Ocean Blvd Ph 4,Boca Raton,FL,33431.0,4020480.0,4.0,4.0,...,-80.068332,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Dovi Ettedgui,The Corcoran Group,Kai Hakim,The Corcoran Group,Cole Devitt,"Premier Estate Properties, Inc",None,None
5,PAST SALE,2025-02-19,Condo/Co-op,3951 N Ocean Blvd #603,Gulf Stream,FL,33483.0,3400000.0,2.0,2.0,...,-80.052356,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Christy Larrimore,Engel & Voelkers Delray Beach,None,None,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group
6,PAST SALE,2025-02-21,Condo/Co-op,170 N Ocean Blvd Unit 510/512,Palm Beach,FL,33480.0,3384250.0,3.0,4.0,...,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,James McCann,"Premier Estate Properties, Inc",Christopher Bode,"Premier Estate Properties, Inc",James McCann,"Premier Estate Properties, Inc",None,None
7,PAST SALE,2025-02-19,Condo/Co-op,101 N Clematis St #402,West Palm Beach,FL,33401.0,2950000.0,3.0,4.5,...,-80.050436,https://www.redfin.com/FL/West-Palm-Beach/101-...,Kristina Gustafson,"Southfields Real Estate, Inc.",Carlos Arellano,"Southfields Real Estate, Inc.",Nicholas Grodzicki,Compass Florida LLC (PB),None,None
8,PAST SALE,2025-02-18,Condo/Co-op,800 Ocean Dr #1004,Juno Beach,FL,33408.0,2007500.0,3.0,2.5,...,-80.052627,https://www.redfin.com/FL/Juno-Beach/800-Ocean...,Cristy Van Den Berg,Waterfront Properties & Club C,None,None,Cristy Van Den Berg,Waterfront Properties & Club C,None,None
9,PAST SALE,2025-02-18,Condo/Co-op,100 Sunrise Ave #219,Palm Beach,FL,33480.0,2000000.0,2.0,2.0,...,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Vicki Copani,Donohue Real Estate Palm Beach,None,None,Dana Landry,Dana E. Landry,None,None


## Current Week's Values

In [23]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

68


In [24]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,036,772


In [25]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$497


In [26]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$70,500,480


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [27]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
87
Input Previous Week Condo Average Sales Price:
382990
Input Previous Week Condo Average PSF:
269
Input Previous Week Condo Sales total (ex: 198_000_000)
33_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-20,Condo/Co-op,1500 Beach Rd #203,Tequesta,FL,33469.0,8980000.0,3.0,4.5,Seaglass,3655.0,NaN,2022.0,NaN,2457.0,6679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Beaches MLS,RX-11003738,N,Y,26.961813,-80.079386,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Nicholas Peterson,Daniel Riordan,None,None,Ryan Billings,Water Pointe Realty Group,None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-20,Condo/Co-op,1500 Beach Rd #203,Tequesta,FL,33469.0,8980000.0,3.0,4.5,Seaglass,3655.0,NaN,2022.0,NaN,2457.0,6679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Beaches MLS,RX-11003738,N,Y,26.961813,-80.079386,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Nicholas Peterson,Daniel Riordan,None,None,Ryan Billings,Water Pointe Realty Group,None,None,1,orange
1,PAST SALE,2025-02-20,Condo/Co-op,130 Sunrise Ave Ph 8,Palm Beach,FL,33480.0,7905500.0,3.0,3.0,Sun And Surf One Hundred And Thirty Condos,2476.0,NaN,1977.0,NaN,3193.0,4493.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Beaches MLS,RX-10977456,N,Y,26.719588,-80.036274,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Spencer Schlager,Douglas Elliman,None,None,Spencer Schlager,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-02-20,Condo/Co-op,2500 S Ocean Blvd #602,Boca Raton,FL,33432.0,6275000.0,6.0,4.5,Luxuria Condo,4313.0,NaN,2009.0,NaN,1455.0,9066.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-11031676,N,Y,26.326626,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,3,blue
3,PAST SALE,2025-02-18,Condo/Co-op,2720 Donald Ross Rd #505,Palm Beach Gardens,FL,33410.0,5600000.0,4.0,4.5,Azure,5205.0,NaN,2018.0,NaN,1076.0,5379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11052872,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Donna Lederman,Waterfront Properties & Club C,None,None,4,blue
4,PAST SALE,2025-02-18,Condo/Co-op,2070 N Ocean Blvd Ph 4,Boca Raton,FL,33431.0,4020480.0,4.0,4.0,Athena Condo,3954.0,NaN,2000.0,NaN,1017.0,4666.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Beaches MLS,RX-11035724,N,Y,26.369827,-80.068332,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Dovi Ettedgui,The Corcoran Group,Kai Hakim,The Corcoran Group,Cole Devitt,"Premier Estate Properties, Inc",None,None,5,blue
5,PAST SALE,2025-02-19,Condo/Co-op,3951 N Ocean Blvd #603,Gulf Stream,FL,33483.0,3400000.0,2.0,2.0,Bellamar House Condo,1420.0,NaN,1979.0,NaN,2394.0,1619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Beaches MLS,RX-11065165,N,Y,26.501091,-80.052356,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Christy Larrimore,Engel & Voelkers Delray Beach,None,None,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,6,blue
6,PAST SALE,2025-02-21,Condo/Co-op,170 N Ocean Blvd Unit 510/512,Palm Beach,FL,33480.0,3384250.0,3.0,4.0,Ocean Towers North Condo,2263.0,NaN,1968.0,NaN,1495.0,3474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11038081,N,Y,26.721061,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,James McCann,"Premier Estate Properties, Inc",Christopher Bode,"Premier Estate Properties, Inc",James McCann,"Premier Estate Properties, Inc",None,None,7,blue
7,PAST SALE,2025-02-19,Condo/Co-op,101 N Clematis St #402,West Palm Beach,FL,33401.0,2950000.0,3.0,4.5,101 Lofts Condo,2205.0,NaN,2001.0,NaN,1338.0,1456.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/101-...,Beaches MLS,RX-11046613,N,Y,26.714014,-80.050436,https://www.redfin.com/FL/West-Palm-Beach/101-...,Kristina Gustafson,"Southfields Real Estate, Inc.",Carlos Arellano,"Southfields Real Estate, Inc.",Nicholas Grodzick

In [35]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [36]:
m.save('index.html')

## Data snagger

In [37]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [38]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-20,Condo/Co-op,1500 Beach Rd #203,Tequesta,FL,33469.0,8980000.0,3.0,4.5,Seaglass,3655.0,NaN,2022.0,NaN,2457.0,6679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Beaches MLS,RX-11003738,N,Y,26.961813,-80.079386,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Nicholas Peterson,Daniel Riordan,None,None,Ryan Billings,Water Pointe Realty Group,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_02242024


# CREATE TEMPLATE 

In [40]:
muni_set = set(df_top_ten['CITY'])

In [41]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [42]:
df_top_ten.reset_index(inplace=True,drop=True)

In [43]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [44]:
top_sale

'Seaglass at 1500 Beach Rd #203 in Tequesta'

In [45]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-20,Condo/Co-op,1500 Beach Rd #203,Tequesta,FL,33469.0,8980000.0,3.0,4.5,Seaglass,3655.0,NaN,2022.0,NaN,2457.0,6679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Beaches MLS,RX-11003738,N,Y,26.961813,-80.079386,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Nicholas Peterson,Daniel Riordan,None,None,Ryan Billings,Water Pointe Realty Group,None,None,1,orange,Seaglass at 1500 Beach Rd #203 in Tequesta
1,PAST SALE,2025-02-20,Condo/Co-op,130 Sunrise Ave Ph 8,Palm Beach,FL,33480.0,7905500.0,3.0,3.0,Sun And Surf One Hundred And Thirty Condos,2476.0,NaN,1977.0,NaN,3193.0,4493.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Beaches MLS,RX-10977456,N,Y,26.719588,-80.036274,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Spencer Schlager,Douglas Elliman,None,None,Spencer Schlager,Douglas Elliman,None,None,2,blue,Sun And Surf One Hundred And Thirty Condos at ...
2,PAST SALE,2025-02-20,Condo/Co-op,2500 S Ocean Blvd #602,Boca Raton,FL,33432.0,6275000.0,6.0,4.5,Luxuria Condo,4313.0,NaN,2009.0,NaN,1455.0,9066.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-11031676,N,Y,26.326626,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,3,blue,Luxuria Condo at 2500 S Ocean Blvd #602 in Boc...
3,PAST SALE,2025-02-18,Condo/Co-op,2720 Donald Ross Rd #505,Palm Beach Gardens,FL,33410.0,5600000.0,4.0,4.5,Azure,5205.0,NaN,2018.0,NaN,1076.0,5379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11052872,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Donna Lederman,Waterfront Properties & Club C,None,None,4,blue,Azure at 2720 Donald Ross Rd #505 in Palm Beac...
4,PAST SALE,2025-02-18,Condo/Co-op,2070 N Ocean Blvd Ph 4,Boca Raton,FL,33431.0,4020480.0,4.0,4.0,Athena Condo,3954.0,NaN,2000.0,NaN,1017.0,4666.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Beaches MLS,RX-11035724,N,Y,26.369827,-80.068332,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Dovi Ettedgui,The Corcoran Group,Kai Hakim,The Corcoran Group,Cole Devitt,"Premier Estate Properties, Inc",None,None,5,blue,Athena Condo at 2070 N Ocean Blvd Ph 4 in Boca...
5,PAST SALE,2025-02-19,Condo/Co-op,3951 N Ocean Blvd #603,Gulf Stream,FL,33483.0,3400000.0,2.0,2.0,Bellamar House Condo,1420.0,NaN,1979.0,NaN,2394.0,1619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Beaches MLS,RX-11065165,N,Y,26.501091,-80.052356,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Christy Larrimore,Engel & Voelkers Delray Beach,None,None,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,6,blue,Bellamar House Condo at 3951 N Ocean Blvd #603...
6,PAST SALE,2025-02-21,Condo/Co-op,170 N Ocean Blvd Unit 510/512,Palm Beach,FL,33480.0,3384250.0,3.0,4.0,Ocean Towers North Condo,2263.0,NaN,1968.0,NaN,1495.0,3474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11038081,N,Y,26.721061,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,James McCann,"Premier Estate Properties, Inc",Christopher Bode,"Premier Estate Properties, Inc",James McCann,"Premier Estate Properties, Inc",None,None,7,blue,Ocean Towers North Condo at 170 N Ocean Blvd U...
7,PAST SALE,2025-02-19,Condo/Co-op,101 N Clematis St #402,West Palm Beach,FL,334

In [48]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Seaglass closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $2,000,000 to $8,980,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, Palm Beach Gardens, Palm Beach, Tequesta, Gulf Stream, West Palm Beach, Juno Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 68 condo sales totaling $70,500,480 million from Feb. 16th to Feb. 23rd. The previous week, brokers closed 87 condo sales totaling $33,300,000.

Last week’s units sold for an average of $1,036,

In [49]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [50]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [51]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [52]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-20,Condo/Co-op,1500 Beach Rd #203,Tequesta,FL,33469.0,8980000.0,3.0,4.5,Seaglass,3655.0,NaN,2022.0,NaN,2457.0,6679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Beaches MLS,RX-11003738,N,Y,26.961813,-80.079386,https://www.redfin.com/FL/Tequesta/1500-Beach-...,Nicholas Peterson,Daniel Riordan,None,None,Ryan Billings,Water Pointe Realty Group,None,None,1,orange,Seaglass at 1500 Beach Rd #203 in Tequesta
1,PAST SALE,2025-02-20,Condo/Co-op,130 Sunrise Ave Ph 8,Palm Beach,FL,33480.0,7905500.0,3.0,3.0,Sun And Surf One Hundred And Thirty Condos,2476.0,NaN,1977.0,NaN,3193.0,4493.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Beaches MLS,RX-10977456,N,Y,26.719588,-80.036274,https://www.redfin.com/FL/Palm-Beach/130-Sunri...,Spencer Schlager,Douglas Elliman,None,None,Spencer Schlager,Douglas Elliman,None,None,2,blue,Sun And Surf One Hundred And Thirty Condos at ...
2,PAST SALE,2025-02-20,Condo/Co-op,2500 S Ocean Blvd #602,Boca Raton,FL,33432.0,6275000.0,6.0,4.5,Luxuria Condo,4313.0,NaN,2009.0,NaN,1455.0,9066.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-11031676,N,Y,26.326626,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Adrienne Jankelow,Coldwell Banker Realty /Delray Beach,None,None,3,blue,Luxuria Condo at 2500 S Ocean Blvd #602 in Boc...
3,PAST SALE,2025-02-18,Condo/Co-op,2720 Donald Ross Rd #505,Palm Beach Gardens,FL,33410.0,5600000.0,4.0,4.5,Azure,5205.0,NaN,2018.0,NaN,1076.0,5379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11052872,N,Y,26.881072,-80.073865,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Donna Lederman,Waterfront Properties & Club C,None,None,4,blue,Azure at 2720 Donald Ross Rd #505 in Palm Beac...
4,PAST SALE,2025-02-18,Condo/Co-op,2070 N Ocean Blvd Ph 4,Boca Raton,FL,33431.0,4020480.0,4.0,4.0,Athena Condo,3954.0,NaN,2000.0,NaN,1017.0,4666.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Beaches MLS,RX-11035724,N,Y,26.369827,-80.068332,https://www.redfin.com/FL/Boca-Raton/2070-N-Oc...,Dovi Ettedgui,The Corcoran Group,Kai Hakim,The Corcoran Group,Cole Devitt,"Premier Estate Properties, Inc",None,None,5,blue,Athena Condo at 2070 N Ocean Blvd Ph 4 in Boca...
5,PAST SALE,2025-02-19,Condo/Co-op,3951 N Ocean Blvd #603,Gulf Stream,FL,33483.0,3400000.0,2.0,2.0,Bellamar House Condo,1420.0,NaN,1979.0,NaN,2394.0,1619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Beaches MLS,RX-11065165,N,Y,26.501091,-80.052356,https://www.redfin.com/FL/Delray-Beach/3951-N-...,Christy Larrimore,Engel & Voelkers Delray Beach,None,None,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,6,blue,Bellamar House Condo at 3951 N Ocean Blvd #603...
6,PAST SALE,2025-02-21,Condo/Co-op,170 N Ocean Blvd Unit 510/512,Palm Beach,FL,33480.0,3384250.0,3.0,4.0,Ocean Towers North Condo,2263.0,NaN,1968.0,NaN,1495.0,3474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,Beaches MLS,RX-11038081,N,Y,26.721061,-80.035205,https://www.redfin.com/FL/Palm-Beach/170-N-Oce...,James McCann,"Premier Estate Properties, Inc",Christopher Bode,"Premier Estate Properties, Inc",James McCann,"Premier Estate Properties, Inc",None,None,7,blue,Ocean Towers North Condo at 170 N Ocean Blvd U...
7,PAST SALE,2025-02-19,Condo/Co-op,101 N Clematis St #402,West Palm Beach,FL,334

In [53]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Seaglass closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $2,000,000 to $8,980,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Boca Raton, Palm Beach Gardens, Palm Beach, Tequesta, Gulf Stream, West Palm Beach, Juno Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 68 condo sales totaling $70,500,480 million from Feb. 16th to Feb. 23rd. The previous week, brokers closed 87 condo sales totaling $33,300,000.

Last week’s units sold for an average of $1,036,

In [54]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Tequesta/1500-Beach-Rd-33469/unit-203/home/42467373


In [55]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/130-Sunrise-Ave-33480/unit-8/home/42434709


In [56]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Palm-Beach/100-Sunrise-Ave-33480/unit-219/home/42434736


In [57]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Juno-Beach/800-Ocean-Dr-33408/unit-1004/home/42398763


In [58]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/130-Sunrise-Ave-33480/unit-8/home/42434709


In [59]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Juno-Beach/800-Ocean-Dr-33408/unit-1004/home/42398763


In [97]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,Four Seasons,2575.0,NaN,2022.0,NaN,2233.0,5298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.130150,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None,1,orange,Four Seasons at 525 N Fort Lauderdale Beach Bl...
1,PAST SALE,2025-02-13,Condo/Co-op,3200 N Ocean Blvd #1809,Fort Lauderdale,FL,33308.0,3200000.0,3.0,3.0,L'hermitage,2845.0,NaN,1998.0,NaN,1125.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10458961,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",Andrew Nowacki,LoKation,None,None,2,blue,L'hermitage at 3200 N Ocean Blvd #1809 in Fort...
2,PAST SALE,2025-02-14,Condo/Co-op,435 Bayshore Dr #301,Fort Lauderdale,FL,33304.0,2950000.0,3.0,3.5,Adagio,2383.0,NaN,2019.0,NaN,1238.0,3694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Beaches MLS,F10483345,N,Y,26.128681,-80.107989,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Tika Van Den Hurk,Douglas Elliman,None,None,Jennifer Gitlan,Coldwell Banker Realty,Jennifer Gitlan,Coldwell Banker Realty,3,blue,Adagio at 435 Bayshore Dr #301 in Fort Lauderdale
3,PAST SALE,2025-02-11,Condo/Co-op,1850 S Ocean Dr #4103,Hallandale Beach,FL,33009.0,1425000.0,3.0,3.0,BEACH CLUB CONDO,1841.0,NaN,2005.0,NaN,774.0,1954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11663132,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Maria Bustamante,Cervera Real Estate Inc,None,None,4,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #4103 in H...
4,PAST SALE,2025-02-12,Condo/Co-op,411 N New River Dr #405,Fort Lauderdale,FL,33301.0,1200000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,736.0,2080.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10465493,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Carlos Martin,Good People Realty,None,None,5,blue,Las Olas Grand at 411 N New River Dr #405 in F...
5,PAST SALE,2025-02-14,Condo/Co-op,1 Las Olas Cir #1104,Fort Lauderdale,FL,33316.0,800000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1971.0,NaN,533.0,1286.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10476190,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,6,blue,Venetian Condominium at 1 Las Olas Cir #1104 i...
6,PAST SALE,2025-02-16,Condo/Co-op,1 Las Olas Cir #1117,Fort Lauderdale,FL,33316.0,710000.0,2.0,2.0,Venetian Condominium,1350.0,NaN,1971.0,NaN,526.0,1199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10448763,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,7,blue,Venetian Condominium at 1 Las Olas Cir #1117 i...
7,PAST SALE,2025-02-12,Condo/Co-op,3020 NE 32nd Ave #908,Fort Lauderdale,FL,33308.0,650000.0,3.0,2.0,TIDES AT BRIDGESIDE SQUAR,1600.0,NaN,2002.0,NaN,406.0,1891.0,Sold,NaN,NaN,https://www.redf

## Time on Market Calculator

In [2]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 7, 12) ## List (Earlier) date
date2 = datetime(2025, 2, 20) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

223


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
chart_df['$/SQUARE FEET'].astype(float)

0    2457.0
1    3193.0
2    1455.0
3    1076.0
4    1017.0
5    2394.0
6    1495.0
7    1338.0
8     994.0
9    1438.0
Name: $/SQUARE FEET, dtype: float64

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [72]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-20,1500 Beach Rd #203 in Tequesta,"$8,980,000",3,4.5,"3,655",2022,"$2,457",Beaches MLS,RX-11003738,Nicholas Peterson Daniel Riordan,Ryan Billings Water Pointe Realty Group
1,2025-02-20,130 Sunrise Ave Ph 8 in Palm Beach,"$7,905,500",3,3.0,"2,476",1977,"$3,193",Beaches MLS,RX-10977456,Spencer Schlager Douglas Elliman,Spencer Schlager Douglas Elliman
2,2025-02-20,2500 S Ocean Blvd #602 in Boca Raton,"$6,275,000",6,4.5,"4,313",2009,"$1,455",Beaches MLS,RX-11031676,Lawrence Mastropieri Mastropieri Group LLC Joy...,Adrienne Jankelow Coldwell Banker Realty /Delr...
3,2025-02-18,2720 Donald Ross Rd #505 in Palm Beach Gardens,"$5,600,000",4,4.5,"5,205",2018,"$1,076",Beaches MLS,RX-11052872,Bradley Ball Frankel Ball Realty LLC Shannon B...,Donna Lederman Waterfront Properties & Club C
4,2025-02-18,2070 N Ocean Blvd Ph 4 in Boca Raton,"$4,020,480",4,4.0,"3,954",2000,"$1,017",Beaches MLS,RX-11035724,Dovi Ettedgui The Corcoran Group Kai Hakim The...,"Cole Devitt Premier Estate Properties, Inc"
5,2025-02-19,3951 N Ocean Blvd #603 in Gulf Stream,"$3,400,000",2,2.0,"1,420",1979,"$2,394",Beaches MLS,RX-11065165,Christy Larrimore Engel & Voelkers Delray Beac...,Candace Friis The Corcoran Group Phillip Friis...
6,2025-02-21,170 N Ocean Blvd Unit 510/512 in Palm Beach,"$3,384,250",3,4.0,"2,263",1968,"$1,495",Beaches MLS,RX-11038081,"James McCann Premier Estate Properties, Inc Ch...","James McCann Premier Estate Properties, Inc"
7,2025-02-19,101 N Clematis St #402 in West Palm Beach,"$2,950,000",3,4.5,"2,205",2001,"$1,338",Beaches MLS,RX-11046613,"Kristina Gustafson Southfields Real Estate, In...",Nicholas Grodzicki Compass Florida LLC (PB)
8,2025-02-18,800 Ocean Dr #1004 in Juno Beach,"$2,007,500",3,2.5,"2,019",1996,$994,Beaches MLS,RX-11034227,Cristy Van Den Berg Waterfront Properties & Cl...,Cristy Van Den Berg Waterfront Properties & Cl...
9,2025-02-18,100 Sunrise Ave #219 in Palm Beach,"$2,000,000",2,2.0,"1,391",1977,"$1,438",Beaches MLS,RX-10950177,Vicki Copani Donohue Real Estate Palm Beach,Dana Landry Dana E. Landry
